In [ ]:
import pandas as pd
import numpy as np
from datetime import *

In [ ]:
# Round up function
def round_up(number):
    if number>0:
        return np.ceil(number)
    else: 
        return np.floor(number)

In [ ]:
# Calculate total function
def calculate_total(Planned_Subtotal,Picking_Subtotal,Scanned,WorkInProgress_Subtotal):
    total=Planned_Subtotal + Picking_Subtotal- Scanned + WorkInProgress_Subtotal
    return total


In [ ]:
# calculate required HC
def calculate_Required_HC(Accumulated_Totals,Hours_of_Processing,Rate,Safety_Buffer):
    Required_HC= round_up(Accumulated_Totals/((1.0-Safety_Buffer)*Hours_of_Processing * Rate))
    return Required_HC

In [ ]:
# Define time calculation scoop 
def define_time_calculation_scoop(calculation_date,times_tab):
    date_time=[]
    date_ref=datetime.strptime(calculation_date+' ' +'23:59', '%Y-%m-%d %H:%M')
    time_ref=date_ref.time()
    min_time=datetime.strptime(times_tab[0],'%H:%M').time()
    max_time=datetime.strptime(times_tab[len(times_tab)-1],'%H:%M').time()
    for i in range(10):
        date_time_obj =datetime.strptime(calculation_date+' ' +times_tab[i], '%Y-%m-%d %H:%M')
        if(date_time_obj.time()>=min_time and date_time_obj.time()<= time_ref):
            date_time.append(date_time_obj)
        else: 
             date_time.append(date_time_obj+ timedelta(days=1)) 
    return date_time

In [ ]:
# Calculate subtotals per process path
def subtotal_function(calculation_date,data,process_path_name,params_tab):
	scanned_per_date=[]
	planned_subtotal_per_date=[]
	picking_subtotal_per_date=[]
	work_in_progress_subtotal_per_date=[]
	date_time=define_time_calculation_scoop(calculation_date,times)
        
	for i in range(len(date_time)):
	    scanned_i=sum(data[(data.ExSD==date_time[i]) & (data.Work_Pool=='Scanned') & (data.Process_Path==process_path_name)].Quantity)
	    scanned_per_date.append(scanned_i)
	    
	    predicted_charge_i=sum(data[(data.ExSD==date_time[i]) & (data.Work_Pool=='PredictedCharge') & (data.Process_Path==process_path_name)].Quantity)
	    planned_subtotal_per_date.append(predicted_charge_i)
	    
	    PickingNotYetPicked_i=sum(data[(data.ExSD==date_time[i]) & (data.Work_Pool=='PickingNotYetPicked') & (data.Process_Path==process_path_name)].Quantity)
	    ReadyToPick_i=sum(data[(data.ExSD==date_time[i]) & (data.Work_Pool=='ReadyToPick') & (data.Process_Path==process_path_name)].Quantity)
	    Picking_i=PickingNotYetPicked_i+ReadyToPick_i
	    picking_subtotal_per_date.append(Picking_i)
	    
	    PickingPicked_i=sum(data[(data.ExSD==date_time[i]) & (data.Work_Pool=='PickingPicked') & (data.Process_Path==process_path_name)].Quantity)
	    Sorted_i=sum(data[(data.ExSD==date_time[i]) & (data.Work_Pool=='Sorted') & (data.Process_Path==process_path_name)].Quantity) 
	    Packing_i=sum(data[(data.ExSD==date_time[i]) & (data.Work_Pool=='Packing') & (data.Process_Path==process_path_name)].Quantity)     
	    Scanned_i=sum(data[(data.ExSD==date_time[i]) & (data.Work_Pool=='Scanned') & (data.Process_Path==process_path_name)].Quantity)
	    ProblemSolving_i=sum(data[(data.ExSD==date_time[i]) & (data.Work_Pool=='ProblemSolving') & (data.Process_Path==process_path_name)].Quantity) 
	    RebinBuffered_i=sum(data[(data.ExSD==date_time[i]) & (data.Work_Pool=='RebinBuffered') & (data.Process_Path==process_path_name)].Quantity) 

	    Work_in_progress_i=PickingPicked_i+Sorted_i+Packing_i+Scanned_i+ProblemSolving_i+RebinBuffered_i
	    work_in_progress_subtotal_per_date.append(Work_in_progress_i)
	    

	subtotals=pd.DataFrame(list(zip(date_time,coef_hours_of_processing,planned_subtotal_per_date,picking_subtotal_per_date, scanned_per_date,work_in_progress_subtotal_per_date))
			       ,columns =['date_time', 'coef_hours_of_processing','planned_subtotal_per_date','picking_subtotal_per_date','scanned_per_date','work_in_progress_subtotal_per_date'])

	subtotals['total']=calculate_total(subtotals.planned_subtotal_per_date,subtotals.picking_subtotal_per_date,subtotals.scanned_per_date,subtotals.work_in_progress_subtotal_per_date)
	series = pd.Series(subtotals.total)
	subtotals['accumulated_totals']= series.cumsum()

	Required_HC_list=[]
	rate=params_tab.loc[params_tab.process_path_name==process_path_name].rate.item()
	subtotals['rate']=rate
	for i in range(subtotals.shape[0]):
	    Required_HC_i=calculate_Required_HC(subtotals.accumulated_totals[i],subtotals.coef_hours_of_processing[i],rate,Safety_Buffer)
	    Required_HC_list.append(Required_HC_i)
	subtotals['Required_HC']=Required_HC_list
	max_required_HC=max(subtotals.loc[0:8,'Required_HC'])
	subtotals['max_required_HC']=max_required_HC
	expected_HC=params_tab.loc[params_tab.process_path_name==process_path_name].PlannedHC.item()
	subtotals['expected_HC']=expected_HC
	delta=expected_HC-max_required_HC
	risk=max_required_HC/expected_HC
	subtotals['delta']=delta
	subtotals['risk']=risk
    
	return subtotals

In [ ]:
def subtotals(calculation_date,data,params_tab):
    subtotals_all=[]
    Process_Path_all_list=data.Process_Path.unique()
    Process_Path_list=Process_Path_all_list[Process_Path_all_list!='PPMultiGiftWrapZone']
    for i in range(len(Process_Path_list)):
        subtotals_all.append(subtotal_function(calculation_date,data,Process_Path_list[i],params_tab))
    return subtotals_all

In [ ]:
# Calculate the risk per process path
def risk_calculator(calculation_date,data,params_tab):
    Process_Path_all_list=data.Process_Path.unique()
    Process_Path_list=Process_Path_all_list[Process_Path_all_list!='PPMultiGiftWrapZone']
    risk_Process_Path_list=[]
    delta_Process_Path_list=[]
    max_required_HC_list=[]
    planned_HC_list=[]
    rate_list=[]
    for i in range(len(Process_Path_list)):
        subtotals_all=subtotals(calculation_date,data,params_tab)
        risk_perc=subtotals_all[i].risk[0]
        risk_Process_Path_list.append(risk_perc)
        delta_Process_Path_list.append(subtotals_all[i].delta[0])
        max_required_HC_list.append(subtotals_all[i].max_required_HC[0])
        rate_list.append(subtotals_all[i].rate[0])
        planned_HC_list.append(subtotals_all[i].expected_HC[0])

    risk_Process_Path=pd.DataFrame(list(zip(Process_Path_list,rate_list,planned_HC_list,max_required_HC_list,delta_Process_Path_list,risk_Process_Path_list)),columns=['Process_Path','rate','planned_HC','required_HC','delta','risk'])
    return risk_Process_Path

In [10]:
# Function to plot the risk map per process path
def risk_map(calculation_date,data,params_tab):
    df=risk_calculator(calculation_date,data,params_tab)
    cm=sns.diverging_palette(-200, 10, l=40,s=80,n=4,as_cmap=True)
    s = df.style.background_gradient(cmap=cm, subset=['risk']).format({'risk': "{:.1%}"}).format({'delta': "{:.2f}"}).format({'required_HC': "{:.2f}"})  
    return s

In [ ]:
# Calculate subtotals per process path
def subtotal_per_process_path(calculation_date,data,params_tab,process_path_name):
    subtotals_all=subtotals(calculation_date,data,params_tab)
    Process_Path_all_list=data.Process_Path.unique()
    Process_Path_list=Process_Path_all_list[Process_Path_all_list!='PPMultiGiftWrapZone']
    output = [idx for idx, element in enumerate(Process_Path_list) if element==process_path_name]
    return subtotals_all[output[0]]

In [ ]:
# Function to write subtotals details per process path to an excel file
def output_file(calculation_date,data,params_tab):
    file_name='risk calculator ' + str(calculation_date)+'.xlsx'
    writer = pd.ExcelWriter(file_name, engine='openpyxl') 
    subtotals_all=subtotals(calculation_date,data,params_tab)
    Process_Path_all_list=data.Process_Path.unique()
    Process_Path_list=Process_Path_all_list[Process_Path_all_list!='PPMultiGiftWrapZone']
    for i in range(len(subtotals_all)):
        l=i*20+2
        subtotals_all[i].to_excel(writer, "Main",index=False,startrow= l, startcol=1) 
        Process_Path_pd=pd.DataFrame([[Process_Path_list[i]]], columns=list(' '))
        Process_Path_pd.to_excel(writer, "Main",index=False,header=False,startrow=l-2 , startcol=0) 
    writer.save()
   

In [11]:
# Function to export the risk map to an excel file
def output_map(calculation_date,data,params_tab):
    file_name='Risk Map ' + str(calculation_date)+'.xlsx'
    writer = pd.ExcelWriter(file_name, engine='openpyxl') 
    risk_map(calculation_date,Exd,params_tab).to_excel(writer, "Main",index=False,header=True ) 
    writer.save()
   